In [172]:
# Import Functions
import pandas as pd
import os
import time
import csv
import json
import fnmatch
import csv
import sqlalchemy as sql
import numpy as np
test1=None
dbHost='mxnunx41'
dbPort='1521'
dbUser='lims'
dbPassword='Mumwit63'
dbName='hwi1'
dburl='oracle://'

In [173]:
# Default Parameters setting
with open('.\configFiles\ElementarConfig.json', 'r') as f:
    config = json.load(f)
lab = config['lab']
analysis = config['analysis']
inst = config['inst']
colsOrder=['Sample.Sample_Number', 'Test.Test_Number','Result.Name', 'Result.Entry', 'Result.Entered_By']

In [174]:
#write the file
def writeFile(dframe,lab,inst,archDestination,importDest,datapath,fname):
    timestr = time.strftime("%Y%m%d%H%M%S")
    FileName = importDest + lab+'_'+inst+'_' + timestr + '.csv'
    dframe.to_csv(FileName, index=False, encoding="utf-8",quotechar='"',quoting=csv.QUOTE_NONNUMERIC) # write  out the file to be imported
    destination = archDestination + fname[:-4] + '.bak' # Backup the processed file
    os.rename(datapath, destination)
    time.sleep(1)

In [175]:
# get Data from DB
def getDataFromDB(sampleNumbersList):
    dburl = 'oracle://'+dbUser+':'+dbPassword+'@'+dbHost+':'+dbPort+'/'+dbName
    engine=sql.create_engine(dburl)
    resultdf=pd.DataFrame()
    sampleNumbrStr=(', '.join("'" + str(item) + "'" for item in sampleNumbersList))
    queryText="SELECT sample_number, analysis, REPLICATE_COUNT,TEST_NUMBER FROM test"
    queryText=queryText+" WHERE SAMPLE_NUMBER IN ("+sampleNumbrStr+") AND analysis IN ('FBR-SR-3A', 'FBR-SR-3C', 'FBR-SR-3E','CODEX-IST', 'CODEX-T')"
    conn = engine.connect()
    resultdf = pd.read_sql(queryText, engine)
    conn.close()
    return resultdf

In [176]:
def extractsuffix(sampleNumstring):
    global test1
    if len(sampleNumstring)>18.0:
        if '/' in sampleNumstring:
            t=sampleNumstring.index("/")+1
            test1=sampleNumstring[t:]
            return test1
        elif '-' in sampleNumstring:
            t=sampleNumstring.index("-")
            analysis=sampleNumstring[:t]
            if(analysis=='CODEX'):
                test1='CODEX'
                return test1
            elif (analysis=='FBR'):
                test1='FBR'
                return test1
    elif len(sampleNumstring)<=9:
        return test1
    elif len(sampleNumstring)==11:
        return test1
    return None

In [177]:
def dbAnalysisSuffix(analysis):
    return (analysis[:analysis.index("-")])

In [178]:
def analysisdef(test):
    analysis = {'IDF': 'CODEX-IST', 'SDF':'CODEX-IST','CODEX':'CODEX-T','FBR':'FBR-SR-3C'}
    return analysis.get(test)

In [179]:
def resultType(resultype):
    resultName = {'IDF': 'IDF % Protein', 'SDF':'SDF % Protein','CODEX':'TDF % Protein','FBR':'% Protein (xtest)'}
    return resultName.get(resultype)

In [180]:
def updateSuffix(sufx):
    if sufx in ('IDF','SDF'):
        return 'CODEX'
    else:
        return sufx

In [181]:
def getreplicate(sampleNumstring):
    if len(sampleNumstring)==11:
        if '/' in sampleNumstring:
            t=sampleNumstring.index("/")+1
            rep=sampleNumstring[t:]
            return rep
    return 1

In [182]:
def formatSampleNumber(sampleNumstring):
    return sampleNumstring[:9]

In [183]:
for eachlab in lab:
    path='//mxns.loc//shares//NA-Instruments//Prod//'+eachlab+'//Import//'+inst+'//'
    archivedDestination='//mxns.loc//shares//NA-Instruments//Prod//'+eachlab+'//Import//'+inst+'//'
    #importDest='//mxns.loc//shares//NA-Instruments//Prod//'+eachlab+'//impf//'
    if (mode == 'T'):
        importDest = '//mxns.loc//shares//NA-Instruments//Prod//' + eachlab + '//testing//'
    else:
        importDest = '//mxns.loc//shares//NA-Instruments//Prod//' + eachlab + '//impf//'
    if os.path.exists(path):
        pass
    else:
        os.makedirs(path)
    if os.path.exists(importDest):
        pass
    else:
        os.makedirs(importDest)
    files = os.listdir(path)
    for filename in files:
        if fnmatch.fnmatch(filename,'*.CSV'):

Lab:Crete
4-23 fbr 422-3_20190424181107.csv
codex 423-1 IST and T._20190425235527.csv
idf424 fbr 424-1 fbr 425-2_20190426151906.csv


In [184]:
#filename='4-23 fbr 422-3_20190424181107.csv'
datapath = path + str(filename)
rawdf = pd.read_csv(datapath,skiprows=1)

In [185]:
rawdf1=rawdf[['Name  ','Protein  [%]']]

In [186]:
rawdf12 = rawdf1.rename(index=str, columns={'Name  ': 'sample', 'Protein  [%]': 'Result'})

In [188]:
rawdf12=rawdf12.dropna()
rawdf12['suffix'] = rawdf12['sample'].apply(extractsuffix)

c:\users\agundlapalli\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [190]:
df_filtered = rawdf12[rawdf12['sample']!='orch lvs	']
df_filtered.fillna(value=pd.np.nan, inplace=True)
df_filtered=df_filtered.dropna()
df_filtered = df_filtered[df_filtered['sample']!='orch lvs'] 

In [192]:
#define the Analysis
#df_filtered['analysis']=df_filtered['suffix'].apply(analysisdef)
df_filtered['resultname']=df_filtered['suffix'].apply(resultType)

In [194]:
df_filtered['replicate_count']=df_filtered['sample'].apply(getreplicate)

In [195]:
dfResult=df_filtered[df_filtered['sample'].map(len) <=11]
dfResult=dfResult[dfResult['sample'].map(len) ==9]

In [196]:
dfResult['sample_number']=dfResult['sample'].apply(formatSampleNumber)
dfResult.drop('sample', axis=1, inplace=True)

In [198]:
dfResult=dfResult.astype({"sample_number": int})
dfResult = dfResult.astype({"replicate_count": int})
sampleNumbersList=list(dfResult['sample_number'].unique())

In [200]:
databasedf=getDataFromDB(sampleNumbersList)

In [201]:
databasedf=databasedf.sort_values(by=['sample_number'])

In [202]:
databasedf['suffix'] = databasedf['analysis'].apply(dbAnalysisSuffix)

In [203]:
dfResult.dtypes

Result             float64
suffix              object
resultname          object
replicate_count      int64
sample_number        int32
dtype: object

In [204]:
dfResult['suffix'] = dfResult['suffix'].apply(updateSuffix)

In [206]:
merged_df = dfResult.merge(databasedf, how = 'inner', on = ['sample_number', 'suffix','replicate_count'])

In [208]:
merged_df['Result.Entered_By'] = 'IMPORTER'
dfResult = merged_df.rename(index=str, 
                           columns={'sample_number': 'Sample.Sample_Number', 'resultname': 'Result.Name','Result': 'Result.Entry','test_number':'Test.Test_Number'})

In [209]:
dfResult=dfResult[colsOrder]

In [210]:
dfResult.head()
dfResult=dfResult.round(3)

In [29]:
#writeFile(dfResult,eachlab,inst,path,importDest,datapath,filename)

In [30]:
path

'//mxns.loc//shares//NA-Instruments//Prod//Crete//Import//Elementar//'